## Implementation of Merge Sort in Julia

In [3]:
using BenchmarkTools

In [4]:
]status BenchmarkTools

Status `~/.julia/environments/v1.5/Project.toml`
  [6e4b80f9] BenchmarkTools v0.7.0


In [5]:
function merge1(A::Array, p::Integer, q::Integer, r::Integer)
  L = A[p:q]
  push!(L, Inf)
  R = A[q+1:r]
  push!(R, Inf)
  i = 1
  j = 1
  for k in p:r
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge1 (generic function with 1 method)

**(?)** Here, which would be better? Using `view` or using copy? When implementing a `MERGE` procedure w/o sentinel, try implement one using `view`.

In [6]:
# Don't use push!
function merge2(A::Array, p::Integer, q::Integer, r::Integer)
  n1 = q - p + 1
  n2 = r - q
  L = zeros(n1+1)
  R = zeros(n2+1)
  L[1:end-1] = A[p:q]
  L[end] = Inf
  R[1:end-1] = A[q+1:r]
  R[end] = Inf
  i = 1
  j = 1
  for k in p:r
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge2 (generic function with 1 method)

In [7]:
typeof(Function)

DataType

In [8]:
merge1 isa Function

true

In [9]:
# Int or other type is better?
#function merge_sort(A::Array, p::Integer, r::Integer; merge::Function=merge2)
function merge_sort(A::Array, p::Integer, r::Integer)
  if p < r
    q = floor((p+r)/2)
    merge_sort(A, p, q)
    merge_sort(A, q+1, r)
    merge2(A, p, q, r)
  end
end

merge_sort (generic function with 1 method)

In [10]:
Int == Int64

true

In [11]:
Int <: Integer

true

In [12]:
Base.show_supertypes(Int)

Int64 <: Signed <: Integer <: Real <: Number <: Any

**(?1)** Actually, the input args `p, q, r` above will always be positive integers (because they represents indices). Wouldn't specifying/restricting
the type declaration in the function argument to `::Unsigned` a better choice?

In [13]:
A = rand(1:100, 10)

10-element Array{Int64,1}:
  82
   4
  16
  54
 100
  50
  61
  15
  73
  35

In [33]:
merge_sort(A, 1, length(A))  # in-place sort
A

10-element Array{Int64,1}:
   4
  15
  16
  35
  50
  54
  61
  73
  82
 100

In [15]:
typeof(length(A))

Int64

In [16]:
merge_sort(A, 1, 10)

LoadError: [91mMethodError: no method matching merge_sort(::Array{Int64,1}, ::Int64, ::Float64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  merge_sort(::Array, ::Integer, [91m::Integer[39m) at In[9]:3[39m

In [17]:
methods(merge_sort)

# 1 method for generic function "merge_sort":
[1] merge_sort(A::Array, p::Integer, r::Integer) in Main at In[9]:3

**(?)** Why the second arg is recognized as a `Float`?<br>
**(R)** After some confusion, I finally realize the cause:
- It's **not** the calls like `merge_sort(A, 1, length(A))` or `merge_sort(A, 1, 10)` which cause problems
- It's rather **the implementation of** `merge_sort` **itself** which causes problems

Indeed, the `q = floor((p+r)/2)` is suspicious. The return value of the function `floor` is probably by default `Float64`.

In [18]:
floor(3.14)

3.0

In [19]:
typeof(floor(3.14))

Float64

Let's correct our def of `merge_sort`. The doc says that we can restrict the return value's type by `floor(T, x)`.<br>
(cf. [https://docs.julialang.org/en/v1/base/math/#Base.floor](https://docs.julialang.org/en/v1/base/math/#Base.floor))

Moreover, let's add to our new function a **keyword argument with default value** `merge::Function=merge2`, giving it the capability to switch its `MERGE` procedure.

However, before that, let's **erase** the old and wrong `merge_sort`.

## Delete A Method: [`Base.delete_method`](https://discourse.julialang.org/t/deleting-methods-in-julia/13455)
```julia
julia> f(x::Integer) = 3.14           
f (generic function with 1 method)

julia> f(x::Int) = 2.71
f (generic function with 2 methods)

julia> subtypes(Integer)
3-element Array{Any,1}:
 Bool
 Signed
 Unsigned

julia> subtypes(Bool)
Type[]

julia> subtypes(Unsigned)
5-element Array{Any,1}:
 UInt128
 UInt16
 UInt32
 UInt64
 UInt8

julia> f(1)
2.71

julia> f(Integer(1))
2.71

julia> f(Unsigned(1))
3.14

julia> m = @which f(1)
f(x::Int64) in Main at REPL[2]:1

julia> m
f(x::Int64) in Main at REPL[2]:1

julia> methods(f)
# 2 methods for generic function "f":
[1] f(x::Int64) in Main at REPL[2]:1
[2] f(x::Integer) in Main at REPL[1]:1

julia> Base.delete_method(m)

julia> methods(f)
# 1 method for generic function "f":
[1] f(x::Integer) in Main at REPL[1]:1
```

In [1]:
?Base.delete_method

```
delete_method(m::Method)
```

Make method `m` uncallable and force recompilation of any methods that use(d) it.


In [22]:
methods(merge_sort)

# 1 method for generic function "merge_sort":
[1] merge_sort(A::Array, p::Integer, r::Integer) in Main at In[9]:3

In [24]:
?Base.delete_method

```
delete_method(m::Method)
```

Make method `m` uncallable and force recompilation of any methods that use(d) it.


In [23]:
Base.delete_method(merge_sort)

LoadError: [91mMethodError: no method matching delete_method(::typeof(merge_sort))[39m
[91m[0mClosest candidates are:[39m
[91m[0m  delete_method([91m::Method[39m) at reflection.jl:1326[39m

**(?3)** `::Method` and `::Function`.

In [25]:
Base.show_supertypes(Method)

Method <: Any

In [26]:
Base.show_supertypes(Function)

Function <: Any

In [27]:
m = @which merge_sort(A, 1, length(A))

merge_sort(A::Array, p::Integer, r::Integer) in Main at In[9]:3

In [28]:
Base.delete_method(m)

In [30]:
methods(merge_sort)

# 0 methods for generic function "merge_sort":

## New, Corrected Definition of `merge_sort`

In [31]:
function merge_sort(A::Array, p::Int, r::Int; merge::Function=merge2)
  if p < r
    q = floor(Integer, (p+r)/2)
    merge_sort(A, p, q)
    merge_sort(A, q+1, r)
    merge(A, p, q, r)
  end
end

merge_sort (generic function with 1 method)

In [32]:
methods(merge_sort)

# 1 method for generic function "merge_sort":
[1] merge_sort(A::Array, p::Int64, r::Int64; merge) in Main at In[31]:1

In [33]:
merge_sort(A, 1, length(A))  # in-place sort
A

10-element Array{Int64,1}:
   4
  15
  16
  35
  50
  54
  61
  73
  82
 100

The reason for which I would like to at least run `merge_sort` once is because the procedure in the book (including its Julia implementation)
looks incredibly and weirdly simple, just a single `while` loop with three lines inside. I want to verify whether it sorts correctly.

## Performance Measure

In [65]:
# Don't use the ∞ sentinel
function merge3(A::Array, p::Integer, q::Integer, r::Integer)
  n1 = q - p + 1
  n2 = r - q
  #L = zeros(n1+1)
  #R = zeros(n2+1)
  L = A[p:q]
  #L[end] = Inf
  R = A[q+1:r]
  #R[end] = Inf
  i = 1
  j = 1
  for k in p:r
    # Must check here whether one of L and R is empty, e.g. whether i or j is out of index range
    if i > n1
      A[k:r] = R[j:end]
      break
    end
    if j > n2
      A[k:r] = L[i:end]
      break
    end
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge3 (generic function with 1 method)

In [66]:
# Don't use the ∞ sentinel and use view()
function merge4(A::Array, p::Integer, q::Integer, r::Integer)
  n1 = q - p + 1
  n2 = r - q
  L = zeros(n1+1)
  R = zeros(n2+1)
  L[1:end-1] = A[p:q]
  L[end] = Inf
  R[1:end-1] = A[q+1:r]
  R[end] = Inf
  i = 1
  j = 1
  for k in p:r
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge4 (generic function with 1 method)